<a href="https://colab.research.google.com/github/zayneeh/Enzyme-engineering/blob/main/Stability_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.3 MB/s eta 0:00:00


In [ ]:
!pip install torch biopython pandas
!pip install fair-esm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/jafetgado/tomer.git
%cd tomer
!pip install -r requirements.txt
!python setup.py install

Cloning into 'tomer'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 117 (delta 58), reused 91 (delta 32), pack-reused 0 (from 0)
Receiving objects: 100% (117/117), 1.76 MiB | 6.52 MiB/s, done.
Resolving deltas: 100% (58/58), done.
/content/tomer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 40.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 64.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord


# Read fasta alignment of the chosen enzymes Q5U780 and P82597 from uniprot.


In [ ]:
def read_fasta_alignment(filename):
    seqs = {}
    current_key = None
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                current_key = line
                seqs[current_key] = ""
            elif line:
                seqs[current_key] += line
    return seqs

# Usage
alignment = read_fasta_alignment("/content/drive/MyDrive/Enzyme Pred/alignment.fasta")

# To view the sequences:
for name, seq in alignment.items():
    print(name)
    print(seq)


>sp|P82597|MGLP_BAC25
---------MSEQY-----PVLSGA-EPFYAENGPVGVLLVHGFTGTPHSMRPLAEAYAKAGYTVCLPRLKGHGTHYEDMERTTFHDWVASVEEGYGWLKQRCQTIFVTG----------------LSMGGTLTLYLAEHHPDICGIVPINAAVDIPAIAAGMTGGGELPRYLDSIGS--------DLKNPDVKELAYEKTPTASLLQLARLMAQTKAKLDRIVCPALIFVSDEDHVVPPGNADIIFQGISSTEK------E-----IVRLRNSYHVATLDY--DQPMII---ERSLE-FFAKHAG--------------------------------------------------------------------------------------------------------------------------------------------------------------
>sp|Q5U780|LIP_BACSP
MKCCRIMFVLLGLWFVFGLSVPGGRTEAASLRANDAPIVLLHGFTGWGRE------------------------------EMFGFKYWGGVRGDIEQWLNDNGYRTYTLAVGPLSSNWDRACEAYAQLVGGTVDYG-AAH-AAKHGHAR--FGRTYPGLLPELKRGGRIHIIAHSQGGQTARMLVSLLENGSQEEREYAKAHNVSLSPLF--------------EGGHHFVLSVTTIATPHDGTTLVNMVDFTDRFFDLQKAVLEAAAVASNVPYTSQVYDFKLDQWGLRRQPGESFDHYFERLKRSPVWTSTDTARYDLSVSGAEKLNQWVQASPNTYYLSFSTERTYRGALTGNHYPELGMNAFSAVVCAPFLGSYRNPTLGIDDRWLENDGIVNTVSMNGPKRGSSDRIVPYDGTLKKGVWNDMGTYNVDHLEIIGVDPNPSFDIRAFYLRLAEQLASLRP


In [ ]:
from collections import OrderedDict

# Let's make sure order is preserved
alignment = OrderedDict(alignment)

seq_names = list(alignment.keys())
seq1 = alignment[seq_names[0]]
seq2 = alignment[seq_names[1]]

variable_positions = []
for i, (aa1, aa2) in enumerate(zip(seq1, seq2)):
    if aa1 != aa2 and aa1 != '-' and aa2 != '-':
        variable_positions.append(i)
print("Variable positions:", variable_positions)


Variable positions: [9, 10, 11, 12, 13, 19, 21, 22, 24, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 46, 47, 48, 49, 81, 82, 83, 85, 86, 88, 89, 90, 91, 92, 93, 94, 95, 96, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 126, 127, 128, 132, 133, 134, 135, 138, 141, 142, 143, 144, 146, 147, 148, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 167, 168, 169, 170, 171, 172, 173, 175, 177, 186, 188, 190, 191, 192, 193, 195, 196, 198, 200, 201, 202, 203, 206, 207, 209, 224, 225, 226, 227, 228, 231, 232, 233, 234, 235, 236, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 253, 254, 261, 267, 269, 270, 271, 272, 273, 275, 276, 277, 278, 279, 281, 286, 287, 288, 289, 293, 294, 296, 297, 299, 301, 302, 303, 304, 305]


In [ ]:
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

In [ ]:
def read_fasta_alignment(filename):
    records = list(SeqIO.parse(filename, "fasta"))
    if len(records) < 2:
        raise ValueError("Alignment must contain at least two sequences.")
    return records

In [ ]:
alignment = read_fasta_alignment("/content/drive/MyDrive/Enzyme Pred/alignment.fasta")
seq1 = str(alignment[0].seq)
seq2 = str(alignment[1].seq)
seq2_no_gaps = seq2.replace("-", "")

##Step 2: Find variable positions in alignment

In [ ]:

variable_positions = []
align_to_seq_idx = {}
seq_idx = 0
for i, (aa1, aa2) in enumerate(zip(seq1, seq2)):
    if aa2 != "-":
        align_to_seq_idx[i] = seq_idx
        seq_idx += 1
    # Only mutate where both sequences have a residue
    if aa1 != aa2 and aa1 != '-' and aa2 != '-':
        variable_positions.append(i)

## Step 3: Generate mutants

In [ ]:

mutants = []
for pos in variable_positions:
    wt_aa = seq2[pos]
    seq_pos = align_to_seq_idx.get(pos)
    if wt_aa == "-" or seq_pos is None:
        continue
    for aa in amino_acids:
        if aa != wt_aa:
            mutant_seq = list(seq2_no_gaps)
            mutant_seq[seq_pos] = aa
            mutant_seq_str = "".join(mutant_seq)
            # FASTA header: indicate mutation, e.g. Q5U780_A123V
            header = f">Q5U780_{wt_aa}{seq_pos+1}{aa}"
            mutants.append(SeqRecord(Seq(mutant_seq_str), id=header, description=""))


## Step 4: Write mutants to FASTA file

In [ ]:

with open("/content/mutants.fasta", "w") as out_f:
    SeqIO.write(mutants, out_f, "fasta")

print(f"{len(mutants)} mutants written to mutants.fasta")

3002 mutants written to mutants.fasta


In [ ]:
num_seqs = sum(1 for _ in SeqIO.parse('/content/mutants.fasta', 'fasta'))
with open('/content/ogts.txt', 'w') as f:
    f.write('OGT\n')   # <-- Header row
    for _ in range(num_seqs):
        f.write('70\n')


#Predict Topt of the mutants using Tomer

In [ ]:
import tomer
df = tomer.pred_fasta_topt('/content/mutants.fasta','/content/ogts.txt' )
print(df)


                ID    Topt   Std err
0      Q5U780_L10A    77.5  1.417921
1      Q5U780_L10C   78.03  1.422424
2      Q5U780_L10D   77.83  1.414217
3      Q5U780_L10E   77.75    1.4124
4      Q5U780_L10F   77.76   1.39908
...            ...     ...       ...
2997  Q5U780_R258S  77.295   1.47328
2998  Q5U780_R258T  77.245  1.469804
2999  Q5U780_R258V  77.205  1.458665
3000  Q5U780_R258W   77.14  1.498801
3001  Q5U780_R258Y   78.52  1.440728

[3002 rows x 3 columns]


## Select the mutants with the highest Topt

In [ ]:
top_candidates = df.sort_values('Topt', ascending=False)
print(top_candidates.head(20))

                ID   Topt   Std err
1659  Q5U780_G144I  79.42  1.270723
44     Q5U780_G12I  79.42  1.270723
2344  Q5U780_V199I  79.42  1.356369
1907  Q5U780_V170I  79.42  1.356369
614    Q5U780_G60I  79.42  1.270723
2686  Q5U780_V231I  79.42  1.356369
1299  Q5U780_R120I  79.42  1.296046
254    Q5U780_R32I  79.42  1.296046
1279  Q5U780_G119I  79.42  1.270723
2477  Q5U780_R207I  79.42  1.296046
595    Q5U780_G59I  79.42  1.270723
2267  Q5U780_G195I  79.42  1.270723
1014   Q5U780_V99I  79.42  1.356369
1242  Q5U780_R117I  79.42  1.296046
2001  Q5U780_G178I  79.42  1.270723
2020  Q5U780_G179I  79.42  1.270723
1033  Q5U780_V103I  79.42  1.356369
443    Q5U780_G48I  79.42  1.270723
634    Q5U780_V61I  79.42  1.356369
653    Q5U780_R62I  79.42  1.296046


In [ ]:
top_ids = set(top_candidates['ID'][:20])

**Save the mutants with the highest topt**

In [ ]:

input_fasta = '/content/mutants.fasta'
output_fasta = '/content/top_mutants.fasta'

# Add ">" to each DataFrame ID to match the FASTA IDs
top_ids = set('>' + i.strip() for i in top_candidates['ID'][:20])

top_records = [rec for rec in SeqIO.parse(input_fasta, 'fasta') if rec.id in top_ids]

print(f"Found {len(top_records)} top sequences.")

SeqIO.write(top_records, output_fasta, 'fasta')
print(f"Saved {len(top_records)} records to {output_fasta}")




Found 20 top sequences.
Saved 20 records to /content/top_mutants.fasta


#Precting the Topt of the enzyme using ESM

In [ ]:
!pip install fair-esm biopython

In [ ]:
from Bio import SeqIO
import torch
import esm
import numpy as np

# Load ESM-2
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()

# Load your FASTA sequences
sequences = [(record.id, str(record.seq)) for record in SeqIO.parse("/content/drive/MyDrive/Enzyme Pred/mutants.fasta", "fasta")]

def get_esm_embeddings(seq_list, batch_size=8):
    all_embeds = []
    for i in range(0, len(seq_list), batch_size):
        batch = seq_list[i:i+batch_size]
        batch_labels, batch_strs, batch_tokens = batch_converter(batch)
        with torch.no_grad():
            results = model(batch_tokens, repr_layers=[6])
        reps = results["representations"][6]
        for j, (_, seq) in enumerate(batch):
            all_embeds.append(reps[j, 1:len(seq)+1].mean(0).cpu().numpy())
    return np.stack(all_embeds)

X_variants = get_esm_embeddings(sequences)


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.linear_model import Ridge

# Example: Load a CSV with columns 'sequence' and 'tm'
train_df = pd.read_csv("/content/drive/MyDrive/Enzyme Pred/train.csv")
train_sequences = [(f"train{i}", seq) for i, seq in enumerate(train_df['protein_sequence'])]
X_train = get_esm_embeddings(train_sequences)
y_train = train_df['tm'].values

# Train the model
reg = Ridge().fit(X_train, y_train)



In [ ]:
tm_pred = reg.predict(X_variants)

In [ ]:

results = []
for rec, pred_tm in zip(seq_records, tm_pred):
    results.append({
        'id': rec.id,
        'sequence': str(rec.seq),
        'predicted_tm': pred_tm
    })

df_results = pd.DataFrame(results)
df_results.to_csv("all_predicted_tm.csv", index=False)

In [ ]:
# Select top mutants with highest Tm
N = 30
top_df = df_results.sort_values(by='predicted_tm', ascending=False).head(N)

In [ ]:
# Save as FASTA
top_seqs = []
id_to_seq = {rec.id: rec for rec in seq_records}
for idx, row in top_df.iterrows():
    rec = id_to_seq[row['id']]
    # Update description to include predicted Tm (optional)
    rec.description = f"predicted_tm={row['predicted_tm']:.2f}"
    top_seqs.append(rec)

SeqIO.write(top_seqs, "top_mutants.fasta", "fasta")

print(f"Saved top {N} mutants with highest predicted Tm to 'top_mutants.fasta'.")
print("All predictions saved to 'all_predicted_tm.csv'.")


**Check the folding and structure of the top variants using Alphafold https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/batch/AlphaFold2_batch.ipynb**